In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

# 该该函数将序列数据分割成样本
def split_sequence(sequence, sw_width, n_features):
    '''
    这个简单的示例，通过for循环实现有重叠截取数据，滑动步长为1，滑动窗口宽度为sw_width。
    以后的文章，会介绍使用yield方法来实现特定滑动步长的滑动窗口的实例。
    '''
    X, y = [], []
    
    for i in range(len(sequence)):
        # 获取单个样本中最后一个元素的索引，因为python切片前闭后开，索引从0开始，所以不需要-1
        end_element_index = i + sw_width
        # 如果样本最后一个元素的索引超过了序列索引的最大长度，说明不满足样本元素个数，则这个样本丢弃
        if end_element_index > len(sequence) - 1:
            break
        # 通过切片实现步长为1的滑动窗口截取数据组成样本的效果
        seq_x, seq_y = sequence[i:end_element_index], sequence[end_element_index]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
        process_X = process_X.reshape((process_X.shape[0], process_X.shape[1], n_features))
    
    print('split_sequence:\nX:\n{}\ny:\n{}\n'.format(np.array(X), np.array(y)))
    print('X_shape:{},y_shape:{}\n'.format(np.array(X).shape, np.array(y).shape))
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    return process_X, process_y

    
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    # 对于一维卷积来说，data_format='channels_last'是默认配置，该API的规则如下：
    # 输入形状为：(batch, steps, channels)；输出形状为：(batch, new_steps, filters)，padding和strides的变化会导致new_steps变化
    # 如果设置为data_format = 'channels_first'，则要求输入形状为： (batch, channels, steps).
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    # 对于一维池化层来说，data_format='channels_last'是默认配置，该API的规则如下：
    # 3D 张量的输入形状为: (batch_size, steps, features)；输出3D张量的形状为：(batch_size, downsampled_steps, features)
    # 如果设置为data_format = 'channels_first'，则要求输入形状为：(batch_size, features, steps)
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    # data_format参数的作用是在将模型从一种数据格式切换到另一种数据格式时保留权重顺序。默认为channels_last。
    # 如果设置为channels_last，那么数据输入形状应为：（batch，…，channels）；如果设置为channels_first，那么数据输入形状应该为（batch，channels，…）
    # 输出为（batch, 之后参数尺寸的乘积）
    model.add(Flatten())
    
    # Dense执行以下操作：output=activation（dot（input，kernel）+bias），
    # 其中,activation是激活函数，kernel是由层创建的权重矩阵，bias是由层创建的偏移向量（仅当use_bias为True时适用）。
    # 2D 输入：(batch_size, input_dim)；对应 2D 输出：(batch_size, units)
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    # 因为要预测下一个时间步的值，因此units设置为1
    model.add(Dense(units=1))
    
    # 配置模型
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    # X为输入数据，y为数据标签；batch_size：每次梯度更新的样本数，默认为32。
    # verbose: 0,1,2. 0=训练过程无输出，1=显示训练过程进度条，2=每训练一个epoch打印一次信息
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    sw_width = 3
    n_features = 1
    epoch_num = 1000
    verbose_set = 0
    
    train_X, train_y = split_sequence(train_seq, sw_width, n_features)

    # 预测
    x_input = np.array([70, 80, 90])
    x_input = x_input.reshape((1, sw_width, n_features))
    
    model, history = oned_cnn_model(sw_width, n_features, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

split_sequence:
X:
[[10 20 30]
 [20 30 40]
 [30 40 50]
 [40 50 60]
 [50 60 70]
 [60 70 80]]
y:
[40 50 60 70 80 90]

X_shape:(6, 3),y_shape:(6,)

train_X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]
train_y:
[40 50 60 70 80 90]

trainX.shape:(6, 3, 1),triany_shape:(6,)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                3250      
______________________

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

def split_sequences(first_seq, secend_seq, sw_width):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        # 切片索引从0开始，区间为前闭后开，所以不用减去1
        end_element_index = i + sw_width
        # 同样的道理，这里考虑最后一个样本正好取到最后一行的数据，那么索引不能减1，如果减去1的话最后一个样本就取不到了。
        if end_element_index > len(dataset):
            break
        
        # 该语句实现步长为1的滑动窗口截取数据功能；
        # 以下切片中，:-1 表示取除最后一列的其他列数据；-1表示取最后一列的数据
        seq_x, seq_y = dataset[i:end_element_index, :-1], dataset[end_element_index-1, -1]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
    
    n_features = process_X.shape[2]
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('n_features:',n_features)
    return process_X, process_y, n_features

    
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=1))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    sw_width = 3

    epoch_num = 1000
    verbose_set = 0
    
    train_X, train_y, n_features = split_sequences(train_seq1, train_seq2, sw_width)

    # 预测
    x_input = np.array([[80, 85], [90, 95], [100, 105]])
    x_input = x_input.reshape((1, sw_width, n_features))
    
    model, history = oned_cnn_model(sw_width, n_features, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]

 [[70 75]
  [80 85]
  [90 95]]]
train_y:
[ 65  85 105 125 145 165 185]

train_X.shape:(7, 3, 2),trian_y.shape:(7,)

n_features: 2
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 64)                0         
_______________________________________

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Input, concatenate
from tensorflow.keras.utils import plot_model

def split_sequences2(first_seq, secend_seq, sw_width, n_features):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        # 切片索引从0开始，区间为前闭后开，所以不用减去1
        end_element_index = i + sw_width
        # 同样的道理，这里考虑最后一个样本正好取到最后一行的数据，那么索引不能减1，如果减去1的话最后一个样本就取不到了。
        if end_element_index > len(dataset):
            break
        
        # 该语句实现步长为1的滑动窗口截取数据功能；
        # 以下切片中，:-1 表示取除最后一列的其他列数据；-1表示取最后一列的数据
        seq_x, seq_y = dataset[i:end_element_index, :-1], dataset[end_element_index-1, -1]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
    
    # [:,:,0]表示三维数组前两个维度的数据全取，第三个维度取第一个数据，可以想象成一摞饼干，取了一块。
    # 本例中 process_X的shape为（7,3,2），所以下式就很好理解了，
    X1 = process_X[:,:,0].reshape(process_X.shape[0], process_X.shape[1], n_features)
    X2 = process_X[:,:,1].reshape(process_X.shape[0], process_X.shape[1], n_features)

    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('X1.shape:{},X2.shape:{}\n'.format(X1.shape, X2.shape))
    
    return X1, X2, process_y

    
def oned_cnn_model(n_steps, n_features, X_1, X_2, y, x1, x2, epoch_num, verbose_set):
    
    visible1 = Input(shape=(n_steps, n_features))
    cnn1 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible1)
    cnn1 = MaxPooling1D(pool_size=2)(cnn1)
    cnn1 = Flatten()(cnn1)

    visible2 = Input(shape=(n_steps, n_features))
    cnn2 = Conv1D(filters=64, kernel_size=2, activation='relu')(visible2)
    cnn2 = MaxPooling1D(pool_size=2)(cnn2)
    cnn2 = Flatten()(cnn2)
    
    merge = concatenate([cnn1, cnn2])
    dense = Dense(50, activation='relu')(merge)
    output = Dense(1)(dense)

    model = Model(inputs=[visible1, visible2], outputs=output)

    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    plot_model(model, to_file='multi_head_cnn_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=200)
    
    history = model.fit([X_1, X_2], y, batch_size=32, epochs=epoch_num, verbose=verbose_set)

    yhat = model.predict([x1,x2], verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    sw_width = 3
    n_features = 1
    epoch_num = 1000
    verbose_set = 0
    
    train_X1, train_X2, train_y = split_sequences2(train_seq1, train_seq2, sw_width, n_features)

    # 预测
    x_input = np.array([[80, 85], [90, 95], [100, 105]])
    x_1 = x_input[:, 0].reshape((1, sw_width, n_features))
    x_2 = x_input[:, 1].reshape((1, sw_width, n_features))
    
    model, history = oned_cnn_model(sw_width, n_features, train_X1, train_X2, train_y, x_1, x_2, epoch_num, verbose_set)

    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]

 [[70 75]
  [80 85]
  [90 95]]]
train_y:
[ 65  85 105 125 145 165 185]

train_X.shape:(7, 3, 2),trian_y.shape:(7,)

X1.shape:(7, 3, 1),X2.shape:(7, 3, 1)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 3, 1)]       0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 3, 1)]       0                               

In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

def split_sequences(first_seq, secend_seq, sw_width):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        end_element_index = i + sw_width
        if end_element_index > len(dataset) - 1:
            break
        
        # 该语句实现步长为1的滑动窗口截取数据功能；
        seq_x, seq_y = dataset[i:end_element_index, :], dataset[end_element_index, :]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
    
    n_features = process_X.shape[2]
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('n_features:',n_features)
    return process_X, process_y, n_features

    
def oned_cnn_model(sw_width, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(sw_width, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=n_features))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    sw_width = 3

    epoch_num = 3000
    verbose_set = 0
    
    train_X, train_y, n_features = split_sequences(train_seq1, train_seq2, sw_width)

    # 预测
    x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, sw_width, n_features))
    
    model, history = oned_cnn_model(sw_width, n_features, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[ 10  15  25]
  [ 20  25  45]
  [ 30  35  65]]

 [[ 20  25  45]
  [ 30  35  65]
  [ 40  45  85]]

 [[ 30  35  65]
  [ 40  45  85]
  [ 50  55 105]]

 [[ 40  45  85]
  [ 50  55 105]
  [ 60  65 125]]

 [[ 50  55 105]
  [ 60  65 125]
  [ 70  75 145]]

 [[ 60  65 125]
  [ 70  75 145]
  [ 80  85 165]]]
train_y:
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]

train_X.shape:(6, 3, 3),trian_y.shape:(6, 3)

n_features: 3
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 2, 64)             448       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 1, 64)             0         
__________________________________

In [15]:
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, Input, concatenate
from tensorflow.keras.utils import plot_model

def split_sequences(first_seq, secend_seq, sw_width):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        end_element_index = i + sw_width
        if end_element_index > len(dataset) - 1:
            break
        
        # 该语句实现步长为1的滑动窗口截取数据功能；
        seq_x, seq_y = dataset[i:end_element_index, :], dataset[end_element_index, :]
        
        X.append(seq_x)
        y.append(seq_y)
        
        process_X, process_y = np.array(X), np.array(y)
    
    n_features = process_X.shape[2]
    y1 = process_y[:, 0].reshape((process_y.shape[0], 1))
    y2 = process_y[:, 1].reshape((process_y.shape[0], 1))
    y3 = process_y[:, 2].reshape((process_y.shape[0], 1))
    
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('n_features:',n_features)
    return process_X, process_y, n_features, y1, y2, y3

    
def oned_cnn_model(n_steps, n_features, X, y, test_X, epoch_num, verbose_set):
    visible = Input(shape=(n_steps, n_features))
    cnn = Conv1D(filters=64, kernel_size=2, activation='relu')(visible)
    cnn = MaxPooling1D(pool_size=2)(cnn)
    cnn = Flatten()(cnn)
    cnn = Dense(50, activation='relu')(cnn)
    
    output1 = Dense(1)(cnn)
    output2 = Dense(1)(cnn)
    output3 = Dense(1)(cnn)
    
    model = Model(inputs=visible, outputs=[output1, output2, output3])
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    plot_model(model, to_file='vector_output_cnn_model.png', show_shapes=True, show_layer_names=True, rankdir='TB', dpi=200)

    model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    sw_width = 3

    epoch_num = 2000
    verbose_set = 0
    
    train_X, train_y, n_features, y1, y2, y3 = split_sequences(train_seq1, train_seq2, sw_width)

    # 预测
    x_input = np.array([[70,75,145], [80,85,165], [90,95,185]])
    x_input = x_input.reshape((1, sw_width, n_features))
    
    model = oned_cnn_model(sw_width, n_features, train_X, [y1, y2, y3], x_input, epoch_num, verbose_set)
    

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[ 10  15  25]
  [ 20  25  45]
  [ 30  35  65]]

 [[ 20  25  45]
  [ 30  35  65]
  [ 40  45  85]]

 [[ 30  35  65]
  [ 40  45  85]
  [ 50  55 105]]

 [[ 40  45  85]
  [ 50  55 105]
  [ 60  65 125]]

 [[ 50  55 105]
  [ 60  65 125]
  [ 70  75 145]]

 [[ 60  65 125]
  [ 70  75 145]
  [ 80  85 165]]]
train_y:
[[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]

train_X.shape:(6, 3, 3),trian_y.shape:(6, 3)

n_features: 3
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 3, 3)]       0                                            
______________________________________________________________________

In [8]:
ss__ = np.array([[101.32952]], dtype=np.float32)

In [13]:
ss__[0][0]

101.32952

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

def split_sequences(sequence, n_steps_in, n_steps_out, n_features):
    '''
    该函数将序列数据分割成样本
    '''

    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        if out_end_ix > len(sequence):
            break

        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
       
        X.append(seq_x)
        y.append(seq_y)
    
    
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], n_features))
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(X, y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(X.shape, y.shape))
    print('n_features:',n_features)
    return X, y

    
def oned_cnn_model(steps_in, n_features, steps_out, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(steps_in, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=steps_out))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    n_steps_in, n_steps_out = 3, 2
    n_features = 1
    epoch_num = 2000
    verbose_set = 0
    
    train_X, train_y = split_sequences(train_seq1, n_steps_in, n_steps_out, n_features)

    # 预测
    x_input = np.array([70, 80, 90])
    x_input = x_input.reshape((1, n_steps_in, n_features))
    
    model, history = oned_cnn_model(n_steps_in, n_features, n_steps_out, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

train_X:
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]]
train_y:
[[40 50]
 [50 60]
 [60 70]
 [70 80]
 [80 90]]

train_X.shape:(5, 3, 1),trian_y.shape:(5, 2)

n_features: 1
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 50)                3250      
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 102       
Total params

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

def split_sequences(first_seq, secend_seq, steps_in, steps_out):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        end_element_index = i + steps_in
        out_end_index = end_element_index + steps_out-1
        
        if end_element_index > len(dataset)-1:
            break
        
        seq_x, seq_y = dataset[i:end_element_index, :-1], dataset[end_element_index-1:out_end_index, -1]
        
        X.append(seq_x)
        y.append(seq_y)
        
    process_X, process_y = np.array(X), np.array(y)
    
    n_features = process_X.shape[2]
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(process_X, process_y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(process_X.shape, process_y.shape))
    print('n_features:',n_features)
    return process_X, process_y, n_features

    
def oned_cnn_model(step_in, step_out, n_features, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(step_in, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=step_out))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    n_steps_in, n_steps_out = 3, 2

    epoch_num = 1000
    verbose_set = 0
    
    train_X, train_y, n_feature = split_sequences(train_seq1, train_seq2, n_steps_in, n_steps_out)

    # 预测
    x_input = np.array([[70, 75], [80, 85], [90, 95]])
    x_input = x_input.reshape((1, n_steps_in, n_feature))
    
    model, history = oned_cnn_model(n_steps_in, n_steps_out, n_feature, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[10 15]
  [20 25]
  [30 35]]

 [[20 25]
  [30 35]
  [40 45]]

 [[30 35]
  [40 45]
  [50 55]]

 [[40 45]
  [50 55]
  [60 65]]

 [[50 55]
  [60 65]
  [70 75]]

 [[60 65]
  [70 75]
  [80 85]]]
train_y:
[[ 65  85]
 [ 85 105]
 [105 125]
 [125 145]
 [145 165]
 [165 185]]

train_X.shape:(6, 3, 2),trian_y.shape:(6, 2)

n_features: 2
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 64)                0         
________________________________

In [10]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPooling1D

def split_sequences(first_seq, secend_seq, steps_in, steps_out):
    '''
    该函数将序列数据分割成样本
    '''
    input_seq1 = np.array(first_seq).reshape(len(first_seq), 1)
    input_seq2 = np.array(secend_seq).reshape(len(secend_seq), 1)
    out_seq = np.array([first_seq[i]+secend_seq[i] for i in range(len(first_seq))])
    out_seq = out_seq.reshape(len(out_seq), 1)
    
    dataset = np.hstack((input_seq1, input_seq2, out_seq))
    print('dataset:\n',dataset)
    
    X, y = [], []
    
    for i in range(len(dataset)):
        end_element_index = i + steps_in
        out_end_index = end_element_index + steps_out
        
        if out_end_index > len(dataset):
            break
        
        seq_x, seq_y = dataset[i:end_element_index, :], dataset[end_element_index:out_end_index, :]
        
        X.append(seq_x)
        y.append(seq_y)
        
    X, y = np.array(X), np.array(y)
    n_output = y.shape[1] * y.shape[2]
    y = y.reshape((y.shape[0], n_output))
    
    n_features = X.shape[2]
    print('train_X:\n{}\ntrain_y:\n{}\n'.format(X, y))
    print('train_X.shape:{},trian_y.shape:{}\n'.format(X.shape, y.shape))
    print('n_features:',n_features)
    print('n_output:',n_output)
    return X, y, n_features, n_output

    
def oned_cnn_model(step_in, step_out, n_features, n_output, X, y, test_X, epoch_num, verbose_set):
    model = Sequential()
    
    model.add(Conv1D(filters=64, kernel_size=2, activation='relu',
                     strides=1, padding='valid', data_format='channels_last',
                     input_shape=(step_in, n_features)))
    
    model.add(MaxPooling1D(pool_size=2, strides=None, padding='valid', 
                           data_format='channels_last')) 
    
    model.add(Flatten())
    
    model.add(Dense(units=50, activation='relu',
                use_bias=True, kernel_initializer='glorot_uniform', bias_initializer='zeros',))
    
    model.add(Dense(units=n_output))
    
    model.compile(optimizer='adam', loss='mse',
                 metrics=['accuracy'], loss_weights=None, sample_weight_mode=None, weighted_metrics=None, target_tensors=None)
    
    print('\n',model.summary())
    
    history = model.fit(X, y, batch_size=32, epochs=epoch_num, verbose=verbose_set)
    
    
    yhat = model.predict(test_X, verbose=0)
    print('\nyhat:', yhat)
    
    return model, history

if __name__ == '__main__':
    
    train_seq1 = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    train_seq2 = [15, 25, 35, 45, 55, 65, 75, 85, 95]
    n_steps_in, n_steps_out = 3, 2

    epoch_num = 7000
    verbose_set = 0
    
    train_X, train_y, n_feature, n_output = split_sequences(train_seq1, train_seq2, n_steps_in, n_steps_out)

    # 预测
    x_input = np.array([[60, 65, 125], [70, 75, 145], [80, 85, 165]])
    x_input = x_input.reshape((1, n_steps_in, n_feature))
    
    model, history = oned_cnn_model(n_steps_in, n_steps_out, n_feature, n_output, train_X, train_y, x_input, epoch_num, verbose_set)
    
    print('\ntrain_acc:%s'%np.mean(history.history['accuracy']), '\ntrain_loss:%s'%np.mean(history.history['loss']))

dataset:
 [[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]
train_X:
[[[ 10  15  25]
  [ 20  25  45]
  [ 30  35  65]]

 [[ 20  25  45]
  [ 30  35  65]
  [ 40  45  85]]

 [[ 30  35  65]
  [ 40  45  85]
  [ 50  55 105]]

 [[ 40  45  85]
  [ 50  55 105]
  [ 60  65 125]]

 [[ 50  55 105]
  [ 60  65 125]
  [ 70  75 145]]]
train_y:
[[ 40  45  85  50  55 105]
 [ 50  55 105  60  65 125]
 [ 60  65 125  70  75 145]
 [ 70  75 145  80  85 165]
 [ 80  85 165  90  95 185]]

train_X.shape:(5, 3, 3),trian_y.shape:(5, 6)

n_features: 3
n_output: 6
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 2, 64)             448       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 1, 64)             0         
_________________________

In [14]:
import sys
print(sys.version)

3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [15]:
import keras

Using TensorFlow backend.


In [16]:
print(keras.__version__)

2.3.1
